## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [39]:
import pandas as pd
import math
import random
from pymystem3 import Mystem
from collections import Counter

data_set = pd.read_csv('/datasets/data.csv')
#print(data_set.info())
#print(data_set.head(10))

### Вывод

База состоит из 21 525 строк и 12 столбцов, занимает более 2 Мбайт памяти. Названия столбцов не требуют изменений. 2 столбца ('days_employed' и 'total_income') содержат по 2 174 пустых значения. По результатам просмотра уникальных значений столбцов, которые содержат качественные характеристики, установлено, что анализу данных должна предшествовать предобработка данных, в ходе которой нужно будет обработать пропуски значений, произвести замену типов данных, обработать дубликаты, категоризировать данные. Кроме того, вызывают вопросы отрицательные значения в столбце 'days_employed' 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [40]:
data_set['children'] = data_set['children'].abs() # Избавимся от отрицательных значений
#Сохраним срез первоначальных данных столбца 'total_income'
data_first_total_income = data_set[['total_income','debt']]
data_first_total_income = data_first_total_income.dropna().reset_index(drop=True)

#Сформируем для столбцов 'days_employed' и 'total_income' выборку данных без пропушенных значений и рассчитаем по ней среднее, количество и стандартное отклонение 
data_notNaN = data_set[data_set['total_income']==data_set['total_income']][['days_employed','total_income']].copy()
stats = data_notNaN.agg(['mean','count','std'])
#print(stats) # Просмотр характеристик данных
#Рассчитаем нижний диапазон 95% доверительного интервала
days_employed_ci95low = stats['days_employed']['mean'] - 1.96 * stats['days_employed']['std']/math.sqrt(stats['days_employed']['count'])
total_income_ci95low = stats['total_income']['mean'] - 1.96 * stats['total_income']['std']/math.sqrt(stats['total_income']['count'])
#print(days_employed_ci95low, total_income_ci95low)
#Создадим функцию, расчитывающую случайное значение данных для отпределенного столбца, наименование которого передается в функцию  
def new_random_value(ci95low, data, column):
    new_value = int(ci95low + random.random()*(2*1.96) * data[column]['std']/math.sqrt(data[column]['count']))
    return new_value
#Заполним пропушенные значения столбов 'days_employed' и 'total_income' случайными значениями, которые генерятся созданной функцией
for row in range(len(data_set['days_employed'])):
    if data_set['days_employed'][row]!=data_set['days_employed'][row]:
        data_set['days_employed'][row] = new_random_value(days_employed_ci95low, stats, 'days_employed')

for row in range(len(data_set['total_income'])):
    if data_set['total_income'][row]!=data_set['total_income'][row]:
        data_set['total_income'][row] = new_random_value(total_income_ci95low, stats, 'total_income')

#print(data_set[['days_employed','total_income']].agg(['mean','count','std'])) # Просмотр новых характеристик данных
data_set['education'] = data_set['education'].str.lower() # Переведем значения столбца 'education' в нижний регистр


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Вывод

2 столбца ('days_employed' и 'total_income') содержат по 2 174 пустых значения. Возможной причиной данной ситуации могло явиться не желание респондентов указывать необходимую информацию. Учитывая, что характеристики наборов данных с пустыми и заполненными ячейками в столбцах 'children' и 'dob_years' отсутствующие значения в столбцах 'days_employed' и 'total_income' будем заполнять случайными значениями из доверительного интервала, сформированного на основании имеющихся данных по этим столбцам.

### Замена типа данных

In [41]:
data_set = data_set.astype({'children':'int8',"days_employed":'int64', 'dob_years':'int8', "education":'category', 'education_id':'int8', 'family_status':'category', 'family_status_id':'int8','gender':'category', 'income_type':'category', 'debt':'int8','total_income':'int64'})
#print(data_set.info())
#print(data_set)

### Вывод

После анализа данных в столбцах, произведена замена их типов, учитывая минимальное и максимальное значения столбцов, содержащих количественные данные, для столбцов, содержащих качественные данные установлен тип 'category'. После проведенных изменений объём занимаемой памяти сократился в 4 раза.  

### Обработка дубликатов

In [42]:
print(data_set.duplicated().sum())

0


### Вывод

После проведенной замены отсутствующих значений на случайные из доверительного интервала дубликаты не выявлены.

### Лемматизация

In [43]:
m = Mystem()
data_set_purpose = data_set['purpose'].unique()
print(data_set_purpose)
text = ' '
for row in range(len(data_set_purpose)):
    text = text + ' ' + data_set_purpose[row]
print(Counter(m.lemmatize(text)))
#print(data_set['purpose'].value_counts())


['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']
Counter({' ': 96, 'покупка': 

### Вывод

В целях лемматизации был сформирован список имевших место быть целей кредитования и подсчитана по нему частота лемм. По результатам анализа целей кредитов просматриваются 4 группы с наиболее часто встремающимися леммами: недвижимость, автомобиль, образование, свадьба.

### Категоризация данных

In [44]:
slovar = {'образование':'образование', 'недвижимость':'недвижимость', 'автомобиль':'автомобиль', 'свадьба':'свадьба', 'жилье':'недвижимость'}

def purpose_type(data): # Функция для выбора категории цели кредита из словаря на основании лемм слов в столбце
    for word in data.split(' '):
        try:
            purpose_type = slovar[m.lemmatize(word)[0]]
        except:
            mask = 1
            #print(data_purpose[row])
    return purpose_type
        
def income_group(total_income):
    if total_income >= 190000: #Значения подобраны для формирования 3-х групп доходов низкий - средний - высокий в пропорции 25% - 50% - 25%
        income_type = 'высокий доход'
    elif total_income <= 110000:
        income_type = 'низкий доход'
    else:
        income_type = 'средний доход'
    return income_type

data_set['purpose_type'] = data_set['purpose'].apply(purpose_type) #Добавление столбца с категорией цели кредита
data_set['income_type'] = data_set['total_income'].apply(income_group) #Добавление столбца с категорией дохода по данным с заполненными пропусками
data_first_total_income['income_type'] = data_first_total_income['total_income'].apply(income_group) #Добавление столбца с категорией дохода по исходным данным

### Вывод

Для категоризации значений столбца 'purpose' были выбраны наиболее часто встречавшиеся цели кредитования: образование, недвижимость, автомобиль, свадьба. Значения с леммой "жилье" отнесены в группу "недвижимость". Значения столбца 'total_income' были распределены в целях оценки на три группы: низкий, средний и высокий.  

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [45]:
data_pivot_children = data_set.pivot_table(index=['children'], values = 'debt', aggfunc=['count','sum',])
data_pivot_children['defolt'] = data_pivot_children['sum'] / data_pivot_children['count'] 
print(data_pivot_children)

          count     sum    defolt
           debt    debt          
children                         
0         14149  1063.0  0.075129
1          4865   445.0  0.091470
2          2055   194.0  0.094404
3           330    27.0  0.081818
4            41     4.0  0.097561
5             9     0.0  0.000000
20           76     8.0  0.105263


### Вывод

Полученные данные свидетельствуют о том, что заёмщики без детей допускали меньше дефолтов по кредитам. При этом количество детей увеличивает уровень дефолтов, отличающееся значение для заёмщиков с тремя детьми может быть вызвано меньшим размером выборки по данным заёмщикам. Наличие заёмщиков с двадцатью детьми требует дополнительной проверки во взаимодействии с подразделением, генерирующим данные. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [46]:
data_pivot_family = data_set.pivot_table(index=['family_status'], values = 'debt', aggfunc=['count','sum',])
data_pivot_family['defolt'] = data_pivot_family['sum'] / data_pivot_family['count'] 
print(data_pivot_family)

                       count    sum    defolt
                        debt   debt          
family_status                                
Не женат / не замужем   2813  274.0  0.097405
в разводе               1195   85.0  0.071130
вдовец / вдова           960   63.0  0.065625
гражданский брак        4177  388.0  0.092890
женат / замужем        12380  931.0  0.075202


### Вывод

Исходя из результатов проведенного анализа, семейное положение оказывает влияние на возврат кредита в срок. Самыми дисциплинированными заёмщиками являются вдовцы/вдовы, самые недисциплинированные - неженатые/ незамужние заёмщики.   

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [47]:
print('Расчет, исходя из данных с заполненными пропусками')
data_pivot_income = data_set.pivot_table(index=['income_type'], values = 'debt', aggfunc=['count','sum',])
data_pivot_income['defolt'] = data_pivot_income['sum'] / data_pivot_income['count'] 
print(data_pivot_income)

print()
print('Расчет показателей по имевшимся данным')
data_pivot_income2 = data_first_total_income.pivot_table(index=['income_type'], values = 'debt', aggfunc=['count','sum',])
data_pivot_income2['defolt'] = data_pivot_income2['sum'] / data_pivot_income2['count'] 
print(data_pivot_income2)



Расчет, исходя из данных с заполненными пропусками
               count    sum    defolt
                debt   debt          
income_type                          
высокий доход   5750  418.0  0.072696
низкий доход    5642  455.0  0.080645
средний доход  10133  868.0  0.085661

Расчет показателей по имевшимся данным
              count  sum    defolt
               debt debt          
income_type                       
высокий доход  5750  418  0.072696
низкий доход   5642  455  0.080645
средний доход  7959  698  0.087699


### Вывод

Уровень дохода также оказывает влияние на вероятность невозврата кредита. При этом, неожиданно, уровень дефолтов у заёмщиков с низким доходом ниже, чем у заёмщиков со средним доходом. При этом значения, полученные по исходным данным без учета пропушенных значений практически не отличались от данных с заполненными пропушенными значениями, что свидетельствует о том, что выбранный метод не оказал существенного влияния на результаты анализа, кроме того, все отсутствовавшие значения вошли в группу со средним доходом. 

- Как разные цели кредита влияют на его возврат в срок?

In [48]:
data_pivot_purpose = data_set.pivot_table(index=['purpose_type'], values='debt', aggfunc=['count','sum',])
data_pivot_purpose['defolt'] = data_pivot_purpose['sum'] / data_pivot_purpose['count'] 
print(data_pivot_purpose)

              count    sum    defolt
               debt   debt          
purpose_type                        
автомобиль     4315  403.0  0.093395
недвижимость  10840  782.0  0.072140
образование    4022  370.0  0.091994
свадьба        2348  186.0  0.079216


### Вывод

Результаты анализа возврата в срок кредита в зависимости от его цели подтвердили то, что наименее рискованным является кредитование недвижимости. Требует дополнительного анализа во взаимодействии с кредитным отделом результат автокредитов с выделением из указанной группы кредитов, по которым гарантией возврата кредита будет являться приобретенный автомобиль.

### Шаг 4. Общий вывод

Уважаемые сотрудники кредитного отдела, спасибо за предоставленные данные. Проведена проверка гипотезы о том, что на возврат кредита в срок могут оказывать влияние семейное положение заёмщика и наличие у него детей. Результаты анализа свидетельствуют о том, что указанные критерии оказывают влияние на возврат кредита в срок. В рамках семейного положения самыми дисциплинированными заёмщиками являются вдовцы/вдовы (уровень дефолтов - 6,56%), самые недисциплинированные - неженатые/ незамужние заёмщики (9,74%). Заёмщики без детей допускали меньше дефолтов по кредитам (7,51%). При этом количество детей увеличивает уровень дефолтов (от 9,14% до 10,52%). Кроме того, уровень дефолтов по кредитам имеет зависимость от дохода (при низком доходе (8,06%), среднем (8,57%), высоком (7,27%)), цели кредитования (кредиты на покупку недвижимости (7,21%), на свадьбу (7,92%), приобретение автомобиля (9,34%), получение образования (9,20%). В ходе анализа, в связи с отсутствием данных в 10% строк в столбце "total_income", отсутствующие значения были заменены на случайные в рамках 95% доверительного интервала, рассчитанного по имеющимся данным, в связи с чем, при наличии заинтересованности, предлагаю совместно обсудить сложивщуюся ситуацию. Надеюсь, что полученные результаты будут полезны в целях построении модели кредитного скоринга. 